# 추천 시스템
## 콘텐츠 기반 필터링
### 참고자료
* https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata
* https://chan-lab.tistory.com/?page=12#google_vignette
* https://www.quora.com/How-does-IMDbs-rating-system-work

In [1]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')
    
movies = pd.read_csv('../data/tmdb_5000_movies.csv')
print(movies.shape)
movies.head()

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


* id
* title : 제목
* genres : 영화 장르
* vote_average : 평균평점
* vote_count : 평점 카운트
* popularity : 인기도
* keywords : 영화의 키워드
* overview : 개요 설명

In [2]:
movies_df = movies[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]

In [3]:
pd.set_option('max_colwidth', 100)
movies_df[['genres', 'keywords']][:1]     # list 안에 dictionary 형식으로 들어가있다. 
## 각 컬럼 width 최대로
#pd.set_option('display.max_colwidth', -1)

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [4]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   title         4803 non-null   object 
 2   genres        4803 non-null   object 
 3   vote_average  4803 non-null   float64
 4   vote_count    4803 non-null   int64  
 5   popularity    4803 non-null   float64
 6   keywords      4803 non-null   object 
 7   overview      4800 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 300.3+ KB


### 텍스트 문자 1차 가공. 파이썬 딕셔너리 변환 후 리스트 형태로 변환

In [5]:
movies_df['genres'].head(1)

0    [{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {...
Name: genres, dtype: object

In [6]:
# ast패키지의 literal_eval 클래스는 텍스트를 자동으로 파싱해서 파이썬 객체로 만들어 준다.
from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)     # 주석처리해도됨

In [7]:
movies_df['genres']

0       [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {...
1            [{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 28, 'name': 'Action'}]
2              [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 80, 'name': 'Crime'}]
3       [{'id': 28, 'name': 'Action'}, {'id': 80, 'name': 'Crime'}, {'id': 18, 'name': 'Drama'}, {'id': ...
4       [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 878, 'name': 'Science Fic...
                                                       ...                                                 
4798            [{'id': 28, 'name': 'Action'}, {'id': 80, 'name': 'Crime'}, {'id': 53, 'name': 'Thriller'}]
4799                                       [{'id': 35, 'name': 'Comedy'}, {'id': 10749, 'name': 'Romance'}]
4800    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}, {'...
4801                        

In [ ]:
movies_df['genres'].head()

In [8]:
# 딕셔너리에 name 키로 가지고 있는 value 추출. 리스트 컴프리헨션으로 
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x ])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x ])     # 주석처리해도됨
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


In [9]:
movies_df['genres']

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4798                        [Action, Crime, Thriller]
4799                                [Comedy, Romance]
4800               [Comedy, Drama, Romance, TV Movie]
4801                                               []
4802                                    [Documentary]
Name: genres, Length: 4803, dtype: object

### 장르 콘텐츠 필터링을 이용한 영화 추천. 장르 문자열을 Count 벡터화 후에 코사인 유사도로 각 영화를 비교
### 장르 문자열의 Count 기반 피처 벡터화

In [11]:
# min_df : min-df는 DF(document-frequency)의 최소 빈도값을 설정해주는 파라미터입니다.
    # DF는 특정 단어가 나타나는 '문서의 수'를 의미합니다.
    # 단어의 수가 아니라 문서의 수
# ngram_range = (1,2)  : 단어의 묶음을 1개부터 2개까지 설정하라
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer 를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환.
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df = 0, ngram_range = (1,2))     # 파라미터에 대해 위의 참고자료 링크보기
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


In [12]:
movies_df['genres_literal']

0       Action Adventure Fantasy Science Fiction
1                       Adventure Fantasy Action
2                         Action Adventure Crime
3                    Action Crime Drama Thriller
4               Action Adventure Science Fiction
                          ...                   
4798                       Action Crime Thriller
4799                              Comedy Romance
4800               Comedy Drama Romance TV Movie
4801                                            
4802                                 Documentary
Name: genres_literal, Length: 4803, dtype: object

### 장르에 따른 영화별 코사인 유사도 추출

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


In [15]:
# 예시
a = np.array([1.5, 0.2, 4.2, 2.5])
print(a)
print('인덱스 정렬:', a.argsort())
print('역순정렬',a[::-1])

[1.5 0.2 4.2 2.5]
인덱스 정렬: [1 0 3 2]
역순정렬 [2.5 4.2 0.2 1.5]


In [20]:
print(genre_sim[:2])
print('---------------------------------')
print(np.argsort(genre_sim[:2]))
#작은 값부터 인덱스 위치 반환
print('---------------------------------')
print([genre_sim[:1][::-1]])
#처음부터 끝까지 -1칸 간격으로 ( == 역순으로)

[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]
---------------------------------
[[2401 3037 3038 ...  813 3494    0]
 [2401 3067 3069 ...  129    1  262]]
---------------------------------
[array([[1.        , 0.59628479, 0.4472136 , ..., 0.        , 0.        ,
        0.        ]])]


In [16]:
# 처음부터 끝가지 -1칸 간격으로( = 역순으로) 정렬
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]       # 역순정렬. 위 셀 참고
print(genre_sim_sorted_ind[:2])

[[   0 3494  813 ... 3038 3037 2401]
 [ 262    1  129 ... 3069 3067 2401]]


### 특정 영화와 장르별 유사도가 높은 영화를 반환하는 함수 생성

In [17]:
def find_sim_movie(df, sorted_ind, title_name, top_n = 10):
    # 인자로 입력된 movies_df df에서 'title'컬럼이 입력된 title_name 값인 df추출
    title_movie = df[df['title'] == title_name]
# sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    #추출된 top_n index들 출력. top_n index 는 2차원 데이터임.
    #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    return df.iloc[similar_indexes]

In [18]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10 )
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


### 평점이 높은 영화 정보 확인

In [21]:
movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending = False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


* 영화 평점(vote)와 관련 평균평점(vote_average) - 불공정
* 평점을 남긴 count(vote_count) 3개 - 다 5점
* vote가 많을수록 평점이 5점이 나올 수 없고, 떨어질 수 있다.
* 참조 
    * https://www.quora.com/How-does-IMDbs-rating-system-work
    <br>
* 평가 횟수에 대한 가중치가 부여된 평점(Weighted Rating) 계산 <br>
         $가중 평점(Weighted Rating) = (v/(v+m)) * R + (m/(v+m)) * C $
* v: 개별 영화에 평점을 투표한 횟수
* m: 평점을 부여하기 위한 최소 투표 횟수(ex.상위 60%, 40%)
* R: 개별 영화에 대한 평균 평점.
* C: 전체 영화에 대한 평균 평점

In [23]:
print(1000/ (1000+300))
print(10/(10+300))

0.7692307692307693
0.03225806451612903


In [24]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6) # 상위 60%
print('C:',round(C,3), 'm:',round(m,3))

C: 6.092 m: 370.2


In [25]:
percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()
def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )   #가중 평점
movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)

In [26]:
movies_df[['title', 'vote_average', 'weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending = False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [27]:
def find_sim_movie(df, sorted_ind, title_name, top_n = 10):
    # 인자로 입력된 movies_df df에서 'title'컬럼이 입력된 title_name 값인 df추출
    title_movie = df[df['title'] == title_name]
# sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index
    title_index = title_movie.index.values
    
    # top_n의 2뱅 해당하는 장르 유사성이 높은 index 추출
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1) 
    
    #기준 영화 index는제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
     
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은순으로 top_n 만큼 추출
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending = False)[:top_n]

In [36]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427


In [39]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'Forrest Gump', 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
3739,Annie Hall,7.8,7.341923
3906,The Apartment,8.1,7.228811
891,Man on the Moon,6.9,6.528592
902,Jerry Maguire,6.7,6.524643
815,Hitch,6.4,6.344006
3262,Enough Said,6.6,6.338237
2549,Where the Heart Is,6.9,6.330582
3989,Lage Raho Munna Bhai,7.6,6.252346
681,The American President,6.5,6.232877
4447,Celeste & Jesse Forever,6.4,6.186275


In [42]:
movies_df[['title']]

,title
0,Avatar
1,Pirates of the Caribbean: At World's End
2,Spectre
3,The Dark Knight Rises
4,John Carter
...,...
4798,El Mariachi
4799,Newlyweds
4800,"Signed, Sealed, Delivered"
4801,Shanghai Calling


In [47]:
movies_df[movies_df['title'] == 'inception']

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal,weighted_vote


In [51]:
movies_df[movies_df['genres_literal'] == 'Action Adventure Fantasy Science Fiction']

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal,weighted_vote
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa...","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",Action Adventure Fantasy Science Fiction,7.166301
14,49521,Man of Steel,"[Action, Adventure, Fantasy, Science Fiction]",6.5,6359,99.398009,"[saving the world, dc comics, superhero, based on comic book, superhuman, alien invasion, reboot...","A young boy learns that he has extraordinary powers and is not of this earth. As a young man, he...",Action Adventure Fantasy Science Fiction,6.477564
46,127585,X-Men: Days of Future Past,"[Action, Adventure, Fantasy, Science Fiction]",7.5,6032,118.078691,"[1970s, mutant, time travel, marvel comic, based on comic book, superhuman, storm, beast, afterc...",The ultimate X-Men ensemble fights a war for the survival of the species across two time periods...,Action Adventure Fantasy Science Fiction,7.418594
813,1924,Superman,"[Action, Adventure, Fantasy, Science Fiction]",6.9,1022,48.507081,"[saving the world, journalist, dc comics, crime fighter, nuclear missile, galaxy, superhero, bas...","Mild-mannered Clark Kent works as a reporter at the Daily Planet alongside his crush, Lois Lane ...",Action Adventure Fantasy Science Fiction,6.685190
870,8536,Superman II,"[Action, Adventure, Fantasy, Science Fiction]",6.5,629,30.515175,"[saving the world, dc comics, sequel, superhero, based on comic book, loss of virginity, crimina...","Three escaped criminals from the planet Krypton test the Man of Steel's mettle. Led by Gen. Zod,...",Action Adventure Fantasy Science Fiction,6.348901
3494,27549,Beastmaster 2: Through the Portal of Time,"[Action, Adventure, Fantasy, Science Fiction]",4.6,17,1.478505,"[based on novel, time travel, sequel, psychotronic, sword and sandal, beastmaster, warrior, time...","Mark Singer returns as Dar, the warrior who can talk to the beasts. Dar is forced to travel to e...",Action Adventure Fantasy Science Fiction,6.026658
